<a href="https://colab.research.google.com/github/ucfilho/Codigos_Teste/blob/master/ABCOptim_plus_Golden_jun_10_2019.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
from random import randint

In [0]:
'''Rosembrock Function'''
def Fun(x):
    fun=0
    a=1.0
    b=100.0
    fun = (a-x[0])**2 + b*(x[1]-x[0]**2)**2
    return fun

In [0]:
''' 
FOBJ gera vetorialmente os valores com a funcao objetivo escalar
em outras palavras gera para populacao o valor da funcao 
a ser otimizada
'''
def FOBJ(X):
    rows = X.shape[0]
    fobj=np.zeros(rows)
    for i in range(rows):
        fobj[i]=Fun(X[i,])
    return fobj

In [0]:
'''Enxame retorna a populacao aleatoria com todos valores entre MIN e MAX'''
def Enxame(PAR,NPAR,MAX,MIN):
    x=np.zeros((NPAR, len(MAX)))
    for j in range(len(MAX)):
        for i in range(NPAR):
            x[i,j]=MIN[j]+(MAX[j]-MIN[j])*np.random.random()
    return x

In [0]:
''' 
Fitness gera vetorialmente os valores com a funcao fitness
equivale a quantidade de nectar da fonte
'''
def FIT(X):
    rows = X.shape[0]
    fit=np.zeros(rows)
    fit=np.copy(FOBJ(X))
    for i in range(rows):
        if(fit[i]>=0):
            fit[i]=1/(fit[i]+1)
        else:
            fit[i]=1+abs(fit[i])
    return fit

In [0]:
'''
probablilidade usada para roleta q atualizacao das onlookers 
semelhante ao genetico
'''

def PROB(x):
    rows=x.shape[0]
    fit=np.zeros(rows)
    prob=np.zeros(rows)
    fit=FIT(x)
    fit_ref=max(fit)
    
    for i in range(rows):
        prob[i] = 0.9*fit[i]/fit_ref+0.1

    return prob

In [0]:
'''
Obtem a melhor escolha greedy search
'''
def GetBest(x,xbest):
    ycal=FOBJ(x)
    best=np.argmin(ycal)
    yref=Fun(xbest)
    if(yref<ycal[best]):
        GBEST=xbest
    else:
        GBEST=x[best,]
    return GBEST
    

In [0]:
'''
employed bee phase
'''
def EmployedBee(xo,x,MAX,MIN,trial) :
    NPAR = xo.shape[0]
    PAR= xo.shape[1]
    V=np.zeros((NPAR, len(MAX)))

    for j in range(PAR):
        for i in range(NPAR):
            fi=np.random.uniform(low=-1.0, high=1.0, size=None)
            rd=randint(0, (NPAR-1))
            V[i,j]=xo[i,j]+(xo[i,j] -x[rd,j])*fi # candidata a solucao
            if (V[i,j]>MAX[j]):
              V[i,j]=np.copy(MAX[j]) # restringe a busca para o intervalo
            if (V[i,j]<MIN[j]):
              V[i,j]=np.copy(MIN[j]) # restringe a busca para o intervalo
              
    YCAL=FOBJ(xo)
    YV=FOBJ(V)
    for i in range(NPAR):
        if(YV[i]<YCAL[i]):
          trial[i]=0
          xo[i,]=np.copy(V[i,])
        else:
          trial[i]=trial[i]+1
            

    return xo,trial

In [0]:
'''
onlooker bee phase
'''
def OnlookerBee(xo,x,trial): #OnlookerBee(xo,x_Employed)
    rows = xo.shape[0]
    cols = xo.shape[1]
    prob=PROB(x)
    prob_ref=max(prob)
    #best=np.argmax(prob)
    
    for i in range(rows):
      prob_ref=np.random.uniform(low=0, high=1.0, size=None)
      rd=randint(0, (NPAR-1))
      if(prob[i] > prob_ref):  
        fi=np.random.uniform(low=-1.0, high=1.0, size=None)
        Xmi=xo[i,]+fi*(xo[i,]-x[rd,])
        xo[i,]=np.copy(Xmi)
        
    YCAL=FOBJ(xo)
    YV=FOBJ(x)
        
    for i in range(rows):
      if(YV[i]<YCAL[i]):
        trial[i]=0
        xo[i,]=np.copy(x[i,])
      else:
        trial[i]=trial[i]+1
    return xo,trial

In [0]:
'''
scout bee phase
'''
#def ScoutBee(trial,x,ntrail,MAX,MIN):
def ScoutBee(x,trial,MAX,MIN):
    PAR = x.shape[0]
    NPAR = x.shape[1]
    for i in range(PAR):
      if(trial[i]>= ntrail):
        trial[i]=0
        for j in range(NPAR):
          x[i,j]=MIN[j]+(MAX[j]-MIN[j])*np.random.random()
    return x,trial

In [0]:

'''
Metodo que alterna e coordena o uso employers,onlooker, scout bees
e greedy search

'''
def ABCOPtim(xo,ITE,PAR,ntrail,NPAR,MAX,MIN):

    trial=np.zeros(NPAR) # inicializa contador p abandonar fonte de alimento
    xbest=Enxame(PAR,1,MAX,MIN)[0,] #inicializa xbest
    #xo=Enxame(PAR,NPAR,MAX,MIN) # inicializa employed bee
    xbest=GetBest(xo,xbest)
    
    for i in range(ITE):
      x=Enxame(PAR,NPAR,MAX,MIN) # comparativo para employed bee
      xo,trial=EmployedBee(xo,x,MAX,MIN,trial)
      xbest=GetBest(xo,xbest)
      x=Enxame(PAR,NPAR,MAX,MIN)  # comparativo para OnlookerBee
      xo,trial=OnlookerBee(xo,x,trial)
      xbest=GetBest(xo,xbest)
      x=Enxame(PAR,NPAR,MAX,MIN) # comparativo para ScoutBee
      xo,trial=ScoutBee(x,trial,MAX,MIN) #  ScoutBee ( a rigor so esta linha basta)
      xbest=GetBest(xo,xbest)
      x=np.copy(xo)
      
    xbest=GetBest(xo,xbest)
    x=np.copy(xo)
    
    return xbest, x,trial  

In [0]:
def GoldenSearch(x,ITE_GOLDEN,MIN,MAX):
  
  
  Num=len(x)
  #print(Num)
  #print(x)
  phi = (1.0 + np.sqrt(5.0))/2.0
  xa=np.copy(x)
  xb=np.copy(x)
  xr=np.zeros(Num)
  xl=np.zeros(Num)

  
  for i in range(Num):
    #delta=(MAX[i]-MIN[i])/10
    delta=x[i]*1.0
    a=delta/phi
    b=delta-a
    xl[i]=x[i]-b
    xr[i]=x[i]+a
    
    # AVALIANDO SE SAIU FORA DO INTERVALO E CORRIGINDO
    if(xr[i]>MAX[i]):
      xr[i]=np.copy(MAX[i])
    if(xl[i]<MIN[i]):
      xl[i]=np.copy(MIN[i])


  for k in range(ITE_GOLDEN):
    for i in range(Num):
      d = (phi - 1)*(xr[i] - xl[i])
      a = xr[i] - d
      b = xl[i] + d
      xa[i]=np.copy(a)
      xb[i]=np.copy(b)
      if(Fun(xb) >= Fun(xa)):
        x[i] = a
        xr[i] = b
      else:
        x[i]= b
        xl[i] = a
      #x[i] = (xr[i]+xl[i])/2

  return x


In [13]:
NPAR=50 #PARTICULAS (fontes de comida)
ITE=200 #ITERACOES
ITE_GOLDEN=50 # quantas vezes vai aplicar GoldenSearch
Num=int(ITE/ITE_GOLDEN) # para executar em blocos bee
#xo=Enxame(PAR,NPAR,MAX,MIN) # inicializa employed bee
PAR=2 #NUM DE PARAMETROS A SER OTIMIZADOS
MAX=[2,2] # MAXIMO DE CADA PARAMETRO
MIN=[-2,-2] # MINIMO DE CADA PARAMETRO
ntrail=25 #numero de buscas ate abandonar uma fonte de alimento
xo=Enxame(PAR,ITE_GOLDEN,MAX,MIN) # inicializa employed bee

for i in range(Num):
    xbest, x,trial =ABCOPtim(xo,ITE,PAR,ntrail,NPAR,MAX,MIN)
    xo=np.copy(x)
    xref=np.copy(xbest)
    #print("vetor",xref)
    xnew=GoldenSearch(xref,ITE_GOLDEN,MIN,MAX)
    #print("vetor",xnew)
    
print("Bee",xbest,"funcao", Fun(xbest),"\n")
print("Bee plus Golden",xnew,"funcao", Fun(xnew),"\n")

print(trial)

    

Bee [1.02169611 1.04064225] funcao 0.0015079993815642766 

Bee plus Golden [1.01852311 1.03218119] funcao 0.0030555680260379242 

[ 1.  1.  1.  0.  1.  0.  0.  0.  0.  1.  2.  0.  3.  1.  0.  1.  0.  0.
  2.  0.  2.  0.  2.  2.  2.  5.  5.  3.  1.  0.  0.  1.  2.  1.  0.  3.
  2.  1.  1.  1.  0.  0.  1.  1.  0.  0.  0.  0.  2. 10.]
